### Generating LAMMPS Input File for a Hydrocarbon Mixture in 2 Tanks Separated by a Nanotube

The notebook generates a LAMMPS input file for a hydrocarbon mixture contained in 2 tanks separated by a nonotube of length 'length' and diameter 'dia'. The inputs accepted by this notebook include:
* `num_mols` = the total number of molecules in each tank
* `dia` = the diameter (angstroms) of the nanotube connecting both tanks
* `length` = the length (angstroms) of the nanotube connecting both tanks
* `filepath` = the directory of the folder containing the necessary input files
* `wall_filename` = the directory of the `csv` file containing the atom coordinates of the tanks
* `angles_filename` = the directory of the `csv` file containing the angles info of the pseudo-atoms in the hydrocarbon mixture
* `bonds_filename` = the directory of the `csv` file containing the bonds info of the the pseudo-atoms in the hydrocarbon mixture
* `dihedrals_filename` = the directory of the `csv` file containing the dihedrals info of the pseudo-atoms in the hydrocarbon mixture
* `mols_filename` = the directory of the `csv` file containing the molecules info of the pseudo-atoms in the hydrocarbon mixture
* `out_filename` = the directory of the output LAMMPS file
* `num_mols` = a list containing the last atom_id of each component in the hydrocarbon mixture, as given in `mols_filename`

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os, sys
import random
from pathlib import Path
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
num_mols = 
dia = 
length = 

In [ ]:
filepath = Path()
wall_filename = filepath/f'60x50nm_mod_with_{dia}nm_hole.csv'
angles_filename = filepath/f'angles_{num_mols}x2.csv'
bonds_filename = filepath/f'bonds_{num_mols}x2.csv'
dihedrals_filename = filepath/f'dihedrals_{num_mols}x2.csv'
mols_filename = filepath/f'mols_{num_mols}x2.csv'
out_filename = filepath/f'{num_mols}x2 mols + dia_{dia}x{length}nm + 2_Tanks LAMMPS_Datafile.txt'

In [ ]:
num_dec = 4

# atom_count is an empty list
atom_count = []

num_mols = []

In [ ]:
# atom_type 1 = C
wall_temp = pd.read_csv(wall_filename)
wall = pd.DataFrame(columns = ['atom_id', 'molecule_id', 'atom_type', 'charge', 'x_pos', 'y_pos', 'z_pos'])

wall.x_pos, wall.y_pos, wall.z_pos = round(wall_temp.x, num_dec), round(wall_temp.y, num_dec), round(wall_temp.z, num_dec)

a_id = range(1, wall.shape[0]+1)
wall.atom_id = a_id
wall.molecule_id = a_id
wall = wall.assign(atom_type = 1)
wall = wall.assign(charge = 0)

del wall_temp

atom_count.append(wall.shape[0])
atom_count.extend(num_mols)

wall = wall.astype({'atom_id': 'int64', 'molecule_id': 'int64', 'atom_type': 'int32', 'charge': 'int32'})

wall

In [ ]:
xlo, xhi = wall.x_pos.min(), wall.x_pos.max()
ylo, yhi = wall.y_pos.min()-5, wall.y_pos.max()+5
zlo, zhi = wall.z_pos.min(), wall.z_pos.max()

In [ ]:
mols = pd.read_csv(mols_filename)
mols = mols.loc[:, ~mols.columns.str.contains('^Unnamed')]

mols.reset_index(drop = True)

mols = mols.astype({'atom_id': 'int64', 'molecule_id': 'int64', 'atom_type': 'int32', 'charge': 'int32'})

mols.atom_id, mols.molecule_id = mols.atom_id + wall.atom_id.max(), mols.molecule_id + wall.molecule_id.max()

mols

In [ ]:
C = pd.concat([wall, mols]).reset_index(drop = True)

del wall, mols

C

In [ ]:
C_bonds = pd.read_csv(bonds_filename)
C_bonds = C_bonds.loc[:, ~C_bonds.columns.str.contains('^Unnamed')]

C_bonds.reset_index(drop = True)

C_bonds.atom_id1, C_bonds.atom_id2 = C_bonds.atom_id1 + atom_count[0], C_bonds.atom_id2 + atom_count[0]

C_bonds

In [ ]:
C_angles = pd.read_csv(angles_filename)
C_angles = C_angles.loc[:, ~C_angles.columns.str.contains('^Unnamed')]

C_angles.reset_index(drop = True)

C_angles.atom_id1, C_angles.atom_id2, C_angles.atom_id3 = C_angles.atom_id1 + atom_count[0], C_angles.atom_id2 + atom_count[0], C_angles.atom_id3 + atom_count[0]

C_angles

In [ ]:
C_dihedrals = pd.read_csv(dihedrals_filename)
C_dihedrals = C_dihedrals.loc[:, ~C_dihedrals.columns.str.contains('^Unnamed')]

C_dihedrals.reset_index(drop = True)

C_dihedrals.atom_id1, C_dihedrals.atom_id2 = C_dihedrals.atom_id1 + atom_count[0], C_dihedrals.atom_id2 + atom_count[0] 
C_dihedrals.atom_id3, C_dihedrals.atom_id4 = C_dihedrals.atom_id3 + atom_count[0], C_dihedrals.atom_id4 + atom_count[0]

C_dihedrals

In [ ]:
new_doc = open(out_filename,'w')

new_doc.write(f'LAMMPS data file. Edited by Mojoola. Atom count {np.cumsum(atom_count)}')

new_doc.write(f'\n\n {C.shape[0]} atoms')
new_doc.write(f'\n {C_bonds.shape[0]} bonds')
new_doc.write(f'\n {C_angles.shape[0]} angles')
new_doc.write(f'\n {C_dihedrals.shape[0]} dihedrals')
new_doc.write(f'\n 0 impropers')

new_doc.write(f'\n\n {len(C.atom_type.unique())} atom types')
new_doc.write(f'\n {len(C_bonds.bond_type.unique())} bond types')
new_doc.write(f'\n {len(C_angles.angle_type.unique())} angle types')
new_doc.write(f'\n {len(C_dihedrals.dihedral_type.unique())} dihedral types')
new_doc.write(f'\n 0 improper types')

new_doc.write(f'\n\n {xlo} {xhi} xlo xhi')
new_doc.write(f'\n {ylo} {yhi} ylo yhi')
new_doc.write(f'\n {zlo} {zhi} zlo zhi')

new_doc.write(f'\n\n Masses\n')
new_doc.write(f'\n 1 12.010700')
new_doc.write(f'\n 2 16.043000')
new_doc.write(f'\n 3 15.035000')
new_doc.write(f'\n 4 14.027000')

new_doc.write(f'\n\n Pair Coeffs\n')
new_doc.write(f'\n 1 0.00241 3.400')
new_doc.write(f'\n 2 0.01275 3.740')
new_doc.write(f'\n 3 0.00844516 3.7500')
new_doc.write(f'\n 4 0.0039639 3.9500')

new_doc.write(f'\n\n Bond Coeffs\n')
new_doc.write(f'\n 1 13.44 1.54')
new_doc.write(f'\n 2 14.35 1.54')
new_doc.write(f'\n 3 15.00 1.54')

new_doc.write(f'\n\n Angle Coeffs\n')
new_doc.write(f'\n 1 2.6929 114')

new_doc.write(f'\n\n Dihedral Coeffs\n')
new_doc.write(f'\n 1 0.061188 -0.011752 0.13638054 0.0')

new_doc.write(f'\n\n Atoms\n\n')
new_doc.write(C.to_string(index = False, header = False, formatters={'atom_id':'{{:<{}}}'.format(C.atom_id.astype(str).str.len().max()).format, 
                                                                     'molecule_id':'{{:<{}}}'.format(C.molecule_id.astype(str).str.len().max()).format,
                                                                     'atom_type':'{{:<{}}}'.format(C.atom_type.astype(str).str.len().max()).format,
                                                                     'charge':'{{:<{}}}'.format(C.charge.astype(str).str.len().max()).format,
                                                                     'x_pos':'{{:<{}}}'.format(C.x_pos.astype(str).str.len().max()).format,
                                                                     'y_pos':'{{:<{}}}'.format(C.y_pos.astype(str).str.len().max()).format,
                                                                     'z_pos':'{{:<{}}}'.format(C.z_pos.astype(str).str.len().max()).format}))

new_doc.write(f'\n\n Bonds\n\n')
new_doc.write(C_bonds.to_string(index = False, header = False, formatters={'bond_id':'{{:<{}}}'.format(C_bonds.bond_id.astype(str).str.len().max()).format, 
                                                                           'bond_type':'{{:<{}}}'.format(C_bonds.bond_type.astype(str).str.len().max()).format,
                                                                           'atom_id1':'{{:<{}}}'.format(C_bonds.atom_id1.astype(str).str.len().max()).format,
                                                                           'atom_id2':'{{:<{}}}'.format(C_bonds.atom_id2.astype(str).str.len().max()).format}))

new_doc.write(f'\n\n Angles\n\n')
new_doc.write(C_angles.to_string(index = False, header = False, formatters={'angle_id':'{{:<{}}}'.format(C_angles.angle_id.astype(str).str.len().max()).format, 
                                                                            'angle_type':'{{:<{}}}'.format(C_angles.angle_type.astype(str).str.len().max()).format,
                                                                            'atom_id1':'{{:<{}}}'.format(C_angles.atom_id1.astype(str).str.len().max()).format,
                                                                            'atom_id2':'{{:<{}}}'.format(C_angles.atom_id2.astype(str).str.len().max()).format,
                                                                            'atom_id3':'{{:<{}}}'.format(C_angles.atom_id3.astype(str).str.len().max()).format}))

new_doc.write(f'\n\n Dihedrals\n\n')
new_doc.write(C_dihedrals.to_string(index = False, header = False, formatters={'dihedral_id':'{{:<{}}}'.format(C_dihedrals.dihedral_id.astype(str).str.len().max()).format, 
                                                                               'dihedral_type':'{{:<{}}}'.format(C_dihedrals.dihedral_type.astype(str).str.len().max()).format,
                                                                               'atom_id1':'{{:<{}}}'.format(C_dihedrals.atom_id1.astype(str).str.len().max()).format,
                                                                               'atom_id2':'{{:<{}}}'.format(C_dihedrals.atom_id2.astype(str).str.len().max()).format,
                                                                               'atom_id3':'{{:<{}}}'.format(C_dihedrals.atom_id3.astype(str).str.len().max()).format,
                                                                               'atom_id4':'{{:<{}}}'.format(C_dihedrals.atom_id4.astype(str).str.len().max()).format}))

new_doc.close()

In [ ]:
z = C.z_pos
x = C.x_pos
y = C.y_pos

ax = plt.axes(projection ="3d")
 
# Creating plot
ax.scatter3D(x, y, z, color = "green")
plt.title("simple 3D scatter plot")
 
# show plot
plt.show()

In [ ]:
C

In [ ]:
np.cumsum(atom_count)